In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Load Data
Load "izquierda diario" dataset from csv.

In [2]:
df = pd.read_csv('./data/izq_econ_news.csv')

In [3]:
df.shape

(144, 7)

In [4]:
df.head(5)

,Unnamed: 0,date,flyer,lead,headline,body,url
0,0,\nLunes 30 de noviembre,SENADO,El objetivo sería “proteger a la Argentina de ...,¿Qué dice la ley de “sostenibilidad de la deud...,Este lunes los senadores dieron media sanción ...,https://www.laizquierdadiario.com/Economia/Que...
1,1,\nLunes 30 de noviembre,ECONOMÍA,"El ministro de Trabajo, Claudio Moroni, confir...",Doble indemnización: pese a no frenar los desp...,El Gobierno nacional prorrogó hasta el próximo...,https://www.laizquierdadiario.com/Economia/Dob...
2,2,\nDomingo 29 de noviembre,AJUSTAN AL SALARIO,El último tramo del aumento se pagaría con el ...,Paritarias estatales de Jujuy: ¿qué cambia con...,El viernes 27/11 por la tarde el Gobierno lueg...,https://www.laizquierdadiario.com/Economia/Par...
3,3,\nSábado 28 de noviembre,JUBILACIONES,El proyecto del Ejecutivo para una nueva ley d...,Escándalo: el Gobierno quiere descontar a los ...,"En un nuevo intento de robo a los jubilados, e...",https://www.laizquierdadiario.com/Economia/Esc...
4,4,\nSábado 28 de noviembre,A PEDIDO DEL FMI,Se oficializa el ataque a las jubilaciones y p...,Ajuste a jubilados: el Gobierno envió al Congr...,El Ejecutivo giró este sábado al Congreso su p...,https://www.laizquierdadiario.com/Economia/Aju...


In [5]:
df.flyer.values

array(['SENADO', 'ECONOMÍA', 'AJUSTAN AL SALARIO ', 'JUBILACIONES',
       'A PEDIDO DEL FMI', 'CONSUMO', 'CIERRE DE MINA AGUILAR S.A.',
       'BRECHA CAMBIARIA', 'INFORME INDEC', 'OPTIMISMO EN LOS “MERCADOS”',
       'INFORME INDEC', 'LEVANTARLA EN PALA',
       'SEMANA ECONÓMICA EN CLAVES', 'INFLACIÓN', 'ESTADOS UNIDOS',
       'INFLACION', 'SOCIEDAD', 'AJUSTE A PEDIDO DEL FMI ',
       'TARIFAS E INFLACIÓN', 'CHANTAJE EMPRESARIAL ', 'TRIBUNA ABIERTA',
       'TUCUMÁN', 'CUMBRE DEL G20', 'PRECIOS', 'COMUNICADO DE PRENSA',
       'DOBLE DISCURSO', 'LICITACIÓN', 'AJUSTE',
       'ESTIMACIÓN JUNTA INTERNA ATE INDEC', 'INFORME INDEC', 'AJUSTE',
       'ANUNCIO OFICIAL', 'EXPORTACIONES', 'DÓLAR Y BRECHA',
       'CELEBRAN LAS PATRONALES AGRARIAS', 'INFORME INDEC', 'AJUSTE',
       'PRESUPUESTO 2021', 'MULTINACIONALES MINERAS',
       'GANANCIA EMPRESARIA', 'NO AL AJUSTE A PEDIDO DEL FMI',
       'COMERCIO INTERNACIONAL', 'MULTINACIONALES MINERAS', 'ACTUALIDAD',
       'ENTREVISTA', 'CONS

### Topics
Find from flyers the different topics inside the section about economy.

In [6]:
df[df.flyer.str.contains('DEUDA|Deuda|BONO', na=False)].shape

(11, 7)

In [7]:
df[df.flyer.str.contains('BRECHA|CAMBI|RESERV', na=False)].shape

(13, 7)

In [8]:
df[df.flyer.str.contains('INFLA|INDEC|SUBA|PRECIO', na=False)].shape

(16, 7)

In [9]:
df[df.flyer.str.contains('FMI|FONDO', na=False)].shape

(5, 7)

In [10]:
df[df.flyer.str.contains('SALARIO|OBRER|TRABAJA', na=False)].shape

(3, 7)

### Corpus (Body)
Get the body of those news which cover topics about inflation and the rate of exchange

In [11]:
df_inflation = df.loc[df.flyer.str.contains('INFLA|INDEC|SUBA|PRECIO', na=False),'body'].values

In [12]:
df_exchange = df.loc[df.flyer.str.contains('BRECHA|CAMBI|RESERV', na=False), 'body'].values

In [13]:
corpus = list(df_inflation) + list(df_exchange)
len(corpus)

29

### Tokenization
#### Manual

In [14]:
import re
import unidecode

import matplotlib.pyplot as plt
from wordcloud import WordCloud

"Stop words", special characters, accents and numbers are removed

In [15]:
stopwords_sp = stopwords.words('spanish')

In [16]:
def tokenizer(texto):

    alphanumeric = re.sub(r'([^\s\w]|_)+', '', texto).lower()
    no_accents = unidecode.unidecode(alphanumeric)
    
    tockens = word_tokenize(no_accents)
    
    tockens_clean = [tocken for tocken in tockens if tocken not in stopwords_sp and tocken.isalpha()]
    
    terminos = tockens_clean

    return terminos

In [17]:
tokenized = [tokenizer(documento) for documento in corpus]

In [18]:
vocabulario = sorted(list(set([word for group in tokenized for word in group])))

In [19]:
len(vocabulario)

2034

#### CountVectorizer

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
vectorizer = CountVectorizer(stop_words = stopwords_sp, lowercase = True, strip_accents='unicode')

In [22]:
cleaning = [re.sub(r'(\d|\$|\%|\+)', '', doc) for doc in corpus]

In [23]:
vectorizer.fit(cleaning)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['eramos', 'estabamos', 'estais', 'estan', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'fueramos', 'fuesemos', 'habeis', 'habia', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'mas', 'mia', 'mias', 'mio', 'mios', 'seais', 'sera', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'si', 'tambien', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 'tendriais', 'tendriamos', 'tendrian', 'tendrias', 'teneis', 'tengais', 'tenia', 'teniais', 'teniamos', 'tenian'

CountVectorizer(stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents='unicode')

In [24]:
len(vectorizer.vocabulary_), len(vectorizer.get_feature_names())

(1936, 1936)

I prefare the manual tokenization results since more terms could be saved.

### Prepare tokens and pickle

In [25]:
def token_matrix_maker(data, vocabulario):
    matriz = np.zeros(shape = (len(data), len(vocabulario)), dtype='int')
    for i, documento in enumerate(data):
        for termino in documento:
            matriz[i, vocabulario.index(termino)] += 1
    return matriz

In [26]:
matrix = token_matrix_maker(tokenized, vocabulario)

In [27]:
len(df_inflation)

16

In [28]:
# make dataframe
token_df = pd.DataFrame(matrix, columns=vocabulario, index = ['doc' + str(i + 1) for i in range(len(tokenized))] )

In [29]:
# add classification column
topics = ['inflation' for i in range(len(df_inflation))] + ['exchange' for i in range(len(df_exchange))]
token_df.insert(loc=0, column='topics', value=topics)

In [30]:
token_df.to_pickle('tokens/topic_tokens_izq.pkl')